You are a data analyst and your client has a large ecommerce company in India (let’s call it X). X gets a thousand orders via their website on a daily basis and they have to deliver them as fast as they can. For delivering the goods ordered by the customers, X has tied up with multiple courier companies in India as delivery partners who charge them some amount per delivery.
The charges are dependent upon two factors:

● Weight of the product   
● Distance between the warehouse (pickup location) and customer’s delivery address (destination location)

On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly basis as charges.
As the amount that X has to pay to the courier companies is very high, they want to verify if the charges levied by their Delivery partners per Order are correct.  

Input Data   
Left Hand Side (LHS) Data (X’s internal data spread across three reports)   
● Website order report- which will list Order IDs and various products (SKUs) part of each order. Order ID is common identifier between X’s order report and courier company invoice   
● SKU master with gross weight of each product-This should be used to calculate total weight of each order and during analysis compare against one reported by courier company in their CSV invoice per Order ID. The courier company calculates weight in slabs of 0.5 KG multiples, so first you have to figure out the total weight of the shipment and then figure out applicable weight slabs.   
For example:    
- If the total weight is 400 gram then weight slab should be 0.5   
- If the total weight is 950 gram then weight slab should be 1   
- If the total weight is 1 KG then weight slab should be 1   
- If the total weight is 2.2 KG then weight slab should be 2.5   

● Warehouse pincode to All India pincode mapping -(this should be used to figure out delivery zone (a/b/c/d/e) and during analysis compare against one reported by courier company in their CSV invoice per Order ID   

RHS Data (courier company invoice in CSV file)   
● Invoice in CSV file mentioning AWB Number (courier company’s own internal ID), Order ID (company X’s order ID), weight of shipment, warehouse pickup pincode, customer delivery pincode, zone of delivery, charges per shipment, type of shipment   
● Courier charges rate card at weight slab and pincode level. If the invoice mentions “Forward charges” then only forward charges (“fwd”) should be applicable as per zone and fixed & additional weights based on weight slabs. If the invoice mentions “Forward and rto charges” then forward charges (“fwd”) and RTO charges (“rto”) should be applicable as per zone and fixed & additional weights based on weight slabs.    
● For the first 0.5 KG, “fixed” rate as per the slab is applicable. For each additional 0.5 KG, “additional” weight in the same proportion is applicable. Total charges will be “fixed” + “total additional” if any   

In [1]:
#importing the required Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#reading tha datasets
order = pd.read_excel('Company X - Order Report.xlsx')
pincode = pd.read_excel("Company X - Pincode Zones.xlsx")
sku = pd.read_excel('Company X - SKU Master.xlsx')
invoice = pd.read_excel("Courier Company - Invoice.xlsx")
rate = pd.read_excel('Courier Company - Rates.xlsx')

In [3]:
output1 = pd.DataFrame()

In [4]:
output1['Order ID'] = invoice['Order ID']
output1['AWB Number'] = invoice['AWB Code']

In [5]:
order=order.merge(sku,how='left',on='SKU')

In [6]:
order.shape

(401, 4)

In [7]:
order['Total weight'] = (order['Order Qty'] * order['Weight (g)'])/1000

In [8]:
# function to round up the weight according to the slab defined
def round1(n):
    x = str(n)
    y = x.index('.')
    num= '0.'+ x[y+1:]
    if (float(num))> 0.5:
        n = np.ceil(n)
    else:
        n = np.floor(n)+np.ceil(n - np.floor(n))/2
    return n

In [9]:
weight_as_per_X =  order.groupby(by=['ExternOrderNo'])[['Total weight']].sum()
weight_as_per_X

,Total weight
ExternOrderNo,
2001806210,0.220
2001806226,0.480
2001806229,0.500
2001806232,1.302
2001806233,0.245
...,...
2001821995,0.477
2001822466,1.376
2001823564,0.672


In [10]:
weight_as_per_X['Weight slab'] = [round1(i) for i in weight_as_per_X['Total weight']]

In [11]:
weight_as_per_X=weight_as_per_X.reset_index()

In [12]:
weight_as_per_X.rename(columns={'ExternOrderNo':'Order ID'},inplace=True)

In [13]:
output1=output1.merge(weight_as_per_X,on='Order ID')

In [14]:
output1=output1.merge(invoice.loc[:,['Order ID','Charged Weight']],on='Order ID')

In [15]:
output1['Weight slab charged by Courier Company (KG)'] = [round1(i) for i in output1['Charged Weight']]

In [16]:
output1

,Order ID,AWB Number,Total weight,Weight slab,Charged Weight,Weight slab charged by Courier Company (KG)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5
1,2001806273,1091117222194,0.615,1.0,1.00,1.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5
3,2001806458,1091117223244,0.700,1.0,1.00,1.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5
...,...,...,...,...,...,...
119,2001812941,1091118551656,0.500,0.5,0.73,1.0
120,2001809383,1091117614452,0.607,1.0,0.50,0.5
121,2001820978,1091120922803,0.515,1.0,0.50,0.5
122,2001811475,1091121844806,0.689,1.0,0.50,0.5


In [17]:
invoice=invoice.merge(pincode,on=['Warehouse Pincode','Customer Pincode'],suffixes=['_charged','_x'])
invoice

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_charged,Type of Shipment,Billing Amount (Rs.),Zone_x
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,d
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,d
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,b
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,d
...,...,...,...,...,...,...,...,...,...
169,1091121846136,2001811305,0.50,121003,302020,d,Forward charges,45.4,b
170,1091121846136,2001811305,0.50,121003,302020,d,Forward charges,45.4,b
171,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,b
172,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,b


In [18]:
output1=output1.merge(invoice.loc[:,['Order ID','Zone_x','Zone_charged','Type of Shipment']],on='Order ID')

In [19]:
#function to calculate the bill according to the perticular weight slab
def rate_calculator(zone,weight,s_type):
    rate_ = 'fwd_'+zone.lower()+'_fixed'
    rate_add = 'fwd_'+zone.lower()+'_additional'
    rto_rate = 'rto_'+zone.lower()+'_fixed'
    rto_rate_add = 'rto_'+zone.lower()+'_additional'
    if s_type == 'Forward charges':
        r = rate[rate_] +(weight-0.5)/0.5 * rate[rate_add]
    else:
        r = (rate[rate_]+rate[rto_rate]) +(weight-0.5)/0.5 * (rate[rate_add]+rate[rto_rate_add])
    return round(r,2)

In [20]:
rate

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [21]:
output1['Expected Charge as per X (Rs.)'] = [rate_calculator(output1['Zone_x'][i],output1['Weight slab'][i],output1['Type of Shipment'][i])[0] for i in output1.index]

In [22]:
output1.drop(['Type of Shipment'],axis = 1,inplace=True)

In [23]:
output1=output1.merge(invoice.loc[:,['Order ID','Billing Amount (Rs.)']],on='Order ID')

In [24]:
# differencing the bills as in charged and expected bill to know that if its over charged or correctly charged or under charged
output1['Difference Between Expected Charges and Billed Charges (Rs.)'] = output1['Expected Charge as per X (Rs.)']-output1['Billing Amount (Rs.)']

In [25]:
output1

,Order ID,AWB Number,Total weight,Weight slab,Charged Weight,Weight slab charged by Courier Company (KG),Zone_x,Zone_charged,Expected Charge as per X (Rs.),Billing Amount (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
353,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1
354,2001811305,1091121846136,0.750,1.0,0.50,0.5,b,d,61.3,45.4,15.9
355,2001811305,1091121846136,0.750,1.0,0.50,0.5,b,d,61.3,45.4,15.9
356,2001811305,1091121846136,0.750,1.0,0.50,0.5,b,d,61.3,45.4,15.9


In [26]:
output1.rename(columns={'Total weight':'Total weight as per X (KG)',
                'Weight slab':'Weight slab as per X (KG)',
               'Charged Weight':'Total weight as per Courier Company (KG)',
               'Zone_x':'Delivery Zone as per X',
               'Zone_charged':'Delivery Zone charged by Courier Company',
               'Billing Amount (Rs.)':'Charges Billed by Courier Company'},inplace=True)

In [27]:
output1

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company,Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.302,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.615,1.0,1.00,1.0,d,d,90.2,90.2,0.0
2,2001806408,1091117222931,2.265,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.700,1.0,1.00,1.0,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.240,0.5,0.15,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
353,2001811475,1091121844806,0.689,1.0,0.50,0.5,e,b,112.1,33.0,79.1
354,2001811305,1091121846136,0.750,1.0,0.50,0.5,b,d,61.3,45.4,15.9
355,2001811305,1091121846136,0.750,1.0,0.50,0.5,b,d,61.3,45.4,15.9
356,2001811305,1091121846136,0.750,1.0,0.50,0.5,b,d,61.3,45.4,15.9


In [28]:
# Now summarising the data on basis of how its charged
output2 = output1.iloc[:,[-1]]

In [29]:
output2['Is_correct'] = np.where(output2 == 0,'Corectly Charged',(np.where(output2>0,'Under Charged','Over Charged')))

In [30]:
output2=output2.groupby(['Is_correct'])[['Difference Between Expected Charges and Billed Charges (Rs.)']].agg(['count','sum'])

In [31]:
output2

Difference Between Expected Charges and Billed Charges (Rs.)  \
                                                                        count   
Is_correct                                                                      
Corectly Charged                                                 28             
Over Charged                                                    298             
Under Charged                                                    32             

                           
                      sum  
Is_correct                 
Corectly Charged      0.0  
Over Charged     -15187.6  
Under Charged      1068.0

In [32]:
output1.to_excel('Output 1.xlsx')

In [33]:
output2.to_excel('Output 2.xlsx')